# Complejidad de los datos

### Introducción

En esta sesión analizaremos el tema de complejidad de los datos y algunas técnicas para tratar los efectos de la misma. Este cuaderno se basa parcialmente en el material del curso de limpieza de datos de Kaggle disponible [aquí](https://www.kaggle.com/learn/data-cleaning).

### Gestión de valores omitidos
Elimine los valores que faltan o rellénelos con un flujo de trabajo automatizado.

La limpieza de datos es una parte clave de la ciencia de datos, pero puede ser muy frustrante. ¿Por qué hay campos de texto ilegibles? ¿Qué hacer con los valores que faltan? ¿Por qué las fechas no tienen el formato correcto? ¿Cómo puede solucionar rápidamente la introducción de datos incoherentes? En este tema, aprenderá por qué se ha encontrado con estos problemas y, lo que es más importante, cómo solucionarlos.

En este cuaderno, aprenderá a abordar algunos de los problemas más comunes de limpieza de datos para que pueda analizar sus datos más rápidamente. Realizará cinco ejercicios prácticos con datos reales y desordenados y responderá a algunas de las preguntas más frecuentes sobre la limpieza de datos.

En este cuaderno, veremos cómo tratar los valores faltantes u omitidos.

#### Primer vistazo a los datos

Lo primero que tenemos que hacer es cargar las bibliotecas y el conjunto de datos que vamos a utilizar.

Para la demostración, utilizaremos un conjunto de datos de eventos ocurridos en partidos de fútbol americano. Debido al tamaño del conjunto de datos, lo descargaremos y posteriormente lo cargaremos a nuestro espacio temporal. [Ir a la página de descarga](https://www.kaggle.com/code/alexisbcook/handling-missing-values/data?select=NFL+Play+by+Play+2009-2017+%28v4%29.csv).

In [ ]:
# módulos que usaremos
import pandas as pd
import numpy as np

# cargamos los datos
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")

# fijamos la semilla para reproducibilidad
np.random.seed(0) 

Lo primero que hay que hacer cuando se recibe un nuevo conjunto de datos es echar un vistazo a algunos de ellos. Esto nos permite ver que todo se lee correctamente y nos da una idea de lo que está pasando con los datos. En este caso, vamos a ver si hay valores perdidos u omitidos, que son representados en Python con `NaN`.

In [ ]:
nfl_data.head()

¿Observamos datos faltantes?

¿Cuántos puntos de datos faltantes tenemos?

Bien, ahora sabemos que tenemos algunos valores faltantes. Veamos cuántos tenemos en cada columna.

In [ ]:
# obtenemos el número de datos faltantes por columna
missing_values_count = nfl_data.isnull().sum()

# Revisamos el número de datos faltantes en las primeras 10 columnas del conjunto de datos (tiene 102 columnas en total).
missing_values_count[0:10]

¿Qué opinas de los resultados mostrados? Sería útil saber qué porcentaje de valores faltan en nuestro conjunto de datos para hacernos una idea más precisa de la magnitud del problema:

In [ ]:
# ¿Cuántos valores faltantes tenemos en total en el conjunto datos?
print(nfl_data.shape)
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# porcentaje de datos faltante
percent_missing = (total_missing/total_cells) * 100
print(f'Celdas totales: {total_cells:,}')   # Se agrega :, a la derecha de la variable para dar formato de miles
print(f'Celdas con datos faltantes: {total_missing:,}') # Se agrega :, a la derecha de la variable para dar formato de miles
print(f'Porcentaje de datos faltantes: {round(percent_missing,2)}%')

¿Qué opinas del porcentaje de datos faltantes?

#### Averiguar por qué faltan datos

Este es el punto en el que entramos en la parte de la ciencia de datos que solemos llamar "intuición de datos", es decir, "analizar realmente los datos e intentar averiguar por qué son como son y cómo afectarán a nuestro análisis". Puede ser una parte frustrante de la ciencia de datos, especialmente si eres nuevo en este campo y no tienes mucha experiencia. Para tratar los valores que faltan, tendrás que usar tu intuición para averiguar por qué falta el valor. Una de las preguntas más importantes que puede hacerse para averiguarlo es la siguiente:

**¿Este valor falta porque no se registró o porque no existe?**

Si falta un valor porque no existe (como la altura del hijo mayor de alguien que no tiene hijos), no tiene sentido intentar adivinar cuál podría ser. Estos valores probablemente quieras mantenerlos como NaN. Por otro lado, si falta un valor porque no se registró, puede intentar adivinar cuál podría haber sido basándose en los demás valores de esa columna y fila. Esto se llama imputación, ¡y aprenderemos a hacerlo a continuación! :)

Veamos un ejemplo. Observando el número de valores faltantes en el marco de datos nfl_data, nos damos cuenta de que la columna "TimesSec" tiene muchos valores faltantes:

In [ ]:
# Revisamos el número de datos faltantes en las primeras 10 columnas del conjunto de datos (tiene 102 columnas en total).
missing_values_count[0:10]

Revisando la documentación, podemos ver que esta columna tiene información sobre el número de segundos que quedaban en el partido cuando se hizo la jugada. Esto significa que estos valores probablemente faltan porque no se registraron, y no porque no existan. Por lo tanto, tendría sentido que intentáramos adivinar cuáles deberían ser en lugar de dejarlos como `NaN`.

Por otra parte, hay otros campos, como "PenalizedTeam", en los que también faltan muchos campos. En este caso, sin embargo, el campo falta porque si no hubo penalización no tiene sentido decir qué equipo fue penalizado. Para esta columna, tendría más sentido dejarla vacía o añadir un tercer valor como "ninguno" y utilizarlo para reemplazar los `NaN`.

Si está realizando un análisis de datos muy cuidadoso, este es el punto en el que miraría cada columna individualmente para averiguar cuál es la mejor estrategia para rellenar los valores que faltan. En el resto de este cuaderno, trataremos algunas técnicas "rápidas y sucias" que pueden ayudarle con los valores que faltan, pero que probablemente acabarán eliminando información útil o añadiendo ruido a los datos.

#### Eliminar valores faltantes
Si tiene prisa o no tiene motivos para averiguar por qué faltan valores, una opción es eliminar las filas o columnas que contengan valores que falten. (Nota: ¡generalmente no se recomienda este enfoque para proyectos importantes! Suele merecer la pena tomarse el tiempo necesario para revisar los datos y examinar una por una todas las columnas con valores faltantes para conocer realmente el conjunto de datos).

Si está seguro de que quiere eliminar las filas con valores faltantes, pandas tiene una función muy útil, dropna() para ayudarle a hacerlo. Vamos a probarla en nuestro conjunto de datos de la NFL.

In [ ]:
# eliminar todos los renglones que contengan un valor faltante u omitido
nfl_data.dropna()

¿Qué sucedió? ¡parece que se han eliminado todos nuestros datos! 😱 Esto se debe a que cada fila de nuestro conjunto de datos tenía al menos un valor faltante. Podríamos tener mejor suerte eliminando todas las columnas que tienen al menos un valor faltante en su lugar.

In [ ]:
# Eliminemos ahora todas las columnas en donde exista un valor faltante
columns_with_na_dropped = nfl_data.dropna(axis=1) #Al agregar el parámetro axis=1 estamos indicando que el criterio sea revisar columnas. Por defecto se revisan renglones (axis=0)
columns_with_na_dropped.head()

In [ ]:
columns_with_na_dropped.shape

Calcular cuanta información hemos perdido

In [ ]:
print(f"Columnas en el conjunto de datos original: {nfl_data.shape[1]}")
print(f"Columnas restantes tras eliminar datos faltantes: {columns_with_na_dropped.shape[1]}")
print(f"Total de columnas eliminadas: {nfl_data.shape[1] - columns_with_na_dropped.shape[1]}")

#### Rellenar automáticamente los valores que faltan
Otra opción es intentar rellenar los valores que faltan. Para ello, vamos a tomar una pequeña subsección de los datos de la NFL para que se imprima bien.

In [ ]:
# obtenemos un pequeño subconjunto del conjunto de datos de la NFL
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

Podemos utilizar la función fillna() de Panda para que rellene por nosotros los valores que faltan en un marco de datos. Una opción que tenemos es especificar con qué queremos que se sustituyan los valores NaN. Aquí, estoy diciendo que me gustaría reemplazar todos los valores NaN con 0.

In [ ]:
# remplazar todos los datos NaN con 0
subset_nfl_data.fillna(0)

Quizás una mejor estrategia sea sustituir los valores que faltan por cualquier valor que le siga directamente en la misma columna. (Esto tiene mucho sentido para conjuntos de datos en los que las observaciones tienen algún tipo de orden lógico).

In [ ]:
# reemplazar todos los NaN el valor que viene directamente después de él en la misma columna
# y sustituir todos los NaN restantes por 0
subset_nfl_data.fillna(method='bfill', axis=0).fillna(0)

Otras opciones para el parámetro `method`:

**ffill**

Rellenar valores propagando la última observación válida a la siguiente válida.

**bfill**

Rellenar valores utilizando la siguiente observación válida para rellenar el hueco.

**interpolate**

Rellena valores NaN usando interpolación.

### Escalado y Normalización
Transformar variables numéricas para que tengan propiedades útiles.

#### Cargamos los módulos a utilizar

In [ ]:
%pip install mlxtend

In [ ]:
# módulos a utilizar
import pandas as pd
import numpy as np

# para transformación Box-Cox
from scipy import stats

# para escalado min_max
from mlxtend.preprocessing import minmax_scaling

# módulos de visualización
import seaborn as sns
import matplotlib.pyplot as plt

# fijamos la semilla para reproducibilidad
np.random.seed(0)

#### Escalado frente a normalización: ¿Cuál es la diferencia?

Una de las razones por las que es fácil confundirse entre escalado y normalización es porque los términos a veces se utilizan indistintamente y, para hacerlo aún más confuso, ¡son muy similares! En ambos casos, se transforman los valores de las variables numéricas para que los puntos de datos transformados tengan propiedades útiles específicas. La diferencia es que en el escalado, se cambia el rango de los datos, mientras que en la normalización, cambia la forma de la distribución de los datos.

Hablemos un poco más en profundidad de cada una de estas opciones.

#### Escalado

Esto significa que está transformando sus datos para que se ajusten a una escala específica, como 0-100 o 0-1. Es conveniente escalar los datos cuando se utilizan métodos basados en medidas de la distancia entre los puntos de datos, como las máquinas de vectores de soporte (SVM) o los vecinos más cercanos (KNN). Con estos algoritmos, un cambio de "1" en cualquier característica numérica recibe la misma importancia.

Por ejemplo, puede consultar los precios de algunos productos en yenes y en dólares estadounidenses. Un dólar estadounidense vale unos 100 yenes, pero si no escala los precios, métodos como SVM o KNN considerarán que una diferencia de precio de 1 yen es tan importante como una diferencia de 1 dólar estadounidense. Está claro que esto no encaja con nuestras intuiciones del mundo. Con la divisa, puedes convertir entre divisas. Pero, ¿qué ocurre con la altura y el peso? No está del todo claro cuántas libras equivalen a una pulgada (o cuántos kilogramos equivalen a un metro).

Al escalar las variables, puedes comparar diferentes variables en igualdad de condiciones. Para ayudarte a entender cómo es el escalado, veamos un ejemplo inventado. (No te preocupes, en el siguiente ejercicio trabajaremos con datos reales).

In [ ]:
# generar 1000 puntos de datos extraídos aleatoriamente de una distribución exponencial
original_data = np.random.exponential(size=1000)

# mix-max escala los datos entre 0 y 1
scaled_data = minmax_scaling(original_data, columns=[0])

# graficamos ambos para comparar
fig, ax = plt.subplots(1, 2, figsize=(15, 3))
sns.histplot(original_data, ax=ax[0], kde=True, legend=False)
ax[0].set_title("Datos Originales")
sns.histplot(scaled_data, ax=ax[1], kde=True, legend=False)
ax[1].set_title("Datos Escalados")
plt.show()

Observe que la forma de los datos no cambia, pero que en lugar de ir de 0 a 8, ahora van de 0 a 1.

#### Normalización
El escalado sólo cambia el rango de los datos. La normalización es una transformación más radical. El objetivo de la normalización es cambiar las observaciones para que puedan describirse como una distribución normal.

[Distribución normal](https://es.wikipedia.org/wiki/Distribuci%C3%B3n_normal): También conocida como "curva de campana", es una distribución estadística específica en la que aproximadamente el mismo número de observaciones se sitúan por encima y por debajo de la media, la media y la mediana son iguales y hay más observaciones cerca de la media. La distribución normal también se conoce como distribución de Gauss.

En general, normalizará sus datos si va a utilizar una técnica de aprendizaje automático o estadística que asuma que sus datos se distribuyen normalmente. Algunos ejemplos son el análisis discriminante lineal (LDA) y el Bayes ingenuo gaussiano. (Consejo profesional: cualquier método con "gaussiano" en el nombre probablemente asume la normalidad).

El método que estamos utilizando para normalizar aquí se llama Transformación [Box-Cox](https://en.wikipedia.org/wiki/Power_transform#Box%E2%80%93Cox_transformation). Echemos un vistazo rápido a cómo se ve la normalización de algunos datos:

In [ ]:
# normalizar los datos exponenciales con boxcox
normalized_data = stats.boxcox(original_data)

# graficamos ambos para comparar
fig, ax=plt.subplots(1, 2, figsize=(15, 3))
sns.histplot(original_data, ax=ax[0], kde=True, legend=False)
ax[0].set_title("Datos Originales")
sns.histplot(normalized_data[0], ax=ax[1], kde=True, legend=False)
ax[1].set_title("Datos Normalizados")
plt.show()

Observe que la forma de nuestros datos ha cambiado. Antes de la normalización tenían casi forma de L. Pero después de la normalización se parecen más al contorno de una campana (de ahí lo de "curva de campana").